In [ ]:
# !pip uninstall -y numpy
# !pip uninstall -y setuptools

# !pip install setuptools
# !pip install numpy

In [1]:
# !wget https://github.com/HSE-LAMBDA/IDAO-2022/archive/refs/heads/main.zip
# !unzip main.zip

from google.colab import drive
drive.mount('/content/drive/')

%cd /content/drive/MyDrive/DLS/IDAO_2022/

# !tar -C "/IDAO_2022/data" -xzvf /content/drive/MyDrive/IDAO_2022/data/dichalcogenides_private.tar.gz
# !tar -C "/IDAO_2022/data" -xzvf /content/drive/MyDrive/IDAO_2022/data/dichalcogenides_public.tar.gz

Mounted at /content/drive/
/content/drive/MyDrive/DLS/IDAO_2022


In [ ]:
!pip install megnet
!pip install pymatgen

In [3]:
import yaml
import json

import pandas as pd
import numpy as np
import tensorflow as tf

from pathlib import Path
from pymatgen.core import Structure
from sklearn.model_selection import train_test_split
from megnet.models import MEGNetModel
from megnet.data.crystal import CrystalGraph

In [4]:
def cation_vacancy(pymatgen_dict: Structure,
                   coord_a: float = 0.041667,
                   coord_b: float = 0.083333):
    vacancy_coords_list = []
    for i in range(8):
        for j in range(8):
            vacancy_coords_list.append([coord_a + 0.125 * i,
                                        coord_b + 0.125 * j,
                                        0.25])
    for i in pymatgen_dict:
        coords = [round(float(i.a), 6), round(float(i.b), 6), round(float(i.c), 6)]
        if coords in vacancy_coords_list:
            vacancy_coords_list.remove(coords)

    for i in range(len(vacancy_coords_list)):
        pymatgen_dict.append('Cr', vacancy_coords_list[i], False)
    return pymatgen_dict


def anion_vacancy(pymatgen_dict: Structure,
                  coord_a: float = 0.083333,
                  coord_b: float = 0.041667,
                  coord_c: float = 0.144826,
                  first_second_layer_distance: float = 0.210348):
    vacancy_coords_list = []
    coords_list_1_layer = []
    coords_list_2_layer = []
    # 1st anion layer
    for i in range(8):
        for j in range(8):
            coords_list_1_layer.append([coord_a + 0.125 * i,
                                        coord_b + 0.125 * j,
                                        coord_c])
    # 2nd anion layer
    for i in range(8):
        for j in range(8):
            coords_list_2_layer.append([coord_a + 0.125 * i,
                                        coord_b + 0.125 * j,
                                        coord_c + first_second_layer_distance])
    # 1st anion layer
    for i in pymatgen_dict:
        coords = [round(float(i.a), 6), round(float(i.b), 6), round(float(i.c), 6)]
        if coords in coords_list_1_layer:
            coords_list_1_layer.remove(coords)

    # 2nd anion layer
    for i in pymatgen_dict:
        coords = [round(float(i.a), 6), round(float(i.b), 6), round(float(i.c), 6)]
        if coords in coords_list_2_layer:
            coords_list_2_layer.remove(coords)

    # 1st anion layer
    for i in range(len(coords_list_1_layer)):
        pymatgen_dict.append('O', coords_list_1_layer[i], False)

    # 2nd anion layer
    for i in range(len(coords_list_2_layer)):
        pymatgen_dict.append('O', coords_list_2_layer[i], False)

    return pymatgen_dict


def data_preprocessing(pymatgen_dict: Structure, cation: int = 0, anion: int = 0):
    formula = str(pymatgen_dict.formula).split(' ')
    for i in formula:
        if 'Mo' in i or 'W' in i:
            cation += int(i.lstrip('MoW'))
        if 'S' in i or 'Se' in i:
            anion += int(i.lstrip('SeS'))
    if cation < 64:
        pymatgen_dict = cation_vacancy(pymatgen_dict)
    if anion < 128:
        pymatgen_dict = anion_vacancy(pymatgen_dict)
    return pymatgen_dict


def read_pymatgen_dict(file):
    with open(file, "r") as f:
        d = json.load(f)
    return data_preprocessing(Structure.from_dict(d))


def energy_within_threshold(prediction, target):
    # compute absolute error on energy per system.
    # then count the no. of systems where max energy error is < 0.02.
    e_thresh = 0.02
    error_energy = tf.math.abs(target - prediction)

    success = tf.math.count_nonzero(error_energy < e_thresh)
    total = tf.size(target)
    return success / tf.cast(total, tf.int64)

def prepare_dataset(dataset_path):
    dataset_path = Path(dataset_path)
    targets = pd.read_csv(dataset_path / "targets.csv", index_col=0)
    struct = {
        item.name.strip(".json"): read_pymatgen_dict(item)
        for item in (dataset_path / "structures").iterdir()
    }

    data = pd.DataFrame(columns=["structures"], index=struct.keys())
    data = data.assign(structures=struct.values(), targets=targets)

    return train_test_split(data, test_size=0.25, random_state=666)


def prepare_model(cutoff, lr):
    nfeat_bond = 100
    r_cutoff = cutoff
    gaussian_centers = np.linspace(0, r_cutoff + 1, nfeat_bond)
    gaussian_width = 0.8

    return MEGNetModel(
        graph_converter=CrystalGraph(cutoff=r_cutoff),
        centers=gaussian_centers,
        width=gaussian_width,
        loss=["MAE"],
        npass=2,
        lr=lr,
        metrics=energy_within_threshold
    )


def main_to_train(config):
    train, test = prepare_dataset(config["datapath"])
    model = prepare_model(
        float(config["model"]["cutoff"]),
        float(config["model"]["lr"]),
    )

    model.train(
        train.structures,
        train.targets,
        validation_structures=test.structures,
        validation_targets=test.targets,
        epochs=int(config["model"]["epochs"]),
        batch_size=int(config["model"]["batch_size"])
    )
    return model


In [ ]:
if __name__ == "__main__":
    with open("config.yaml") as file:
        config = yaml.safe_load(file)
    main_to_train(config)

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/initializers.py:120: UserWarning: The initializer GlorotUniform is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initializer instance more than once.
  warnings.warn(


Epoch 1/800
1/1 [==============================] - 3s 3s/step


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Выходные данные были обрезаны до нескольких последних строк (5000).
35/35 [==============================] - 13s 372ms/step - loss: 0.0234 - energy_within_threshold: 0.6801
Epoch 323/800
35/35 [==============================] - 13s 367ms/step - loss: 0.0236 - energy_within_threshold: 0.6790
Epoch 324/800
35/35 [==============================] - 13s 368ms/step - loss: 0.0226 - energy_within_threshold: 0.6951
Epoch 325/800
35/35 [==============================] - 13s 367ms/step - loss: 0.0249 - energy_within_threshold: 0.6751
Epoch 326/800
35/35 [==============================] - 13s 359ms/step - loss: 0.0242 - energy_within_threshold: 0.6862
Epoch 327/800
35/35 [==============================] - 13s 366ms/step - loss: 0.0238 - energy_within_threshold: 0.6979
Epoch 328/800
35/35 [==============================] - 13s 369ms/step - loss: 0.0244 - energy_within_threshold: 0.6830
Epoch 329/800
35/35 [==============================] - 13s 366ms/step - loss: 0.0236 - energy_within_threshold: 0

In [ ]:
def main_to_predict(config):
    train, test = prepare_dataset(config["datapath"])

    model = prepare_model(
        float(config["model"]["cutoff"]), float(config["model"]["lr"])
    )
    #? model.load_weights(config['checkpoint_path'])

    # dataset_path = Path(config['test_datapath'])
    # struct = {item.name.strip('.json'): read_pymatgen_dict(item) for item in (dataset_path/'structures').iterdir()}
    struct = test

    # private_test = pd.DataFrame(columns=['id', 'structures'], index=struct.keys())
    # private_test = private_test.assign(structures=struct.values())
    # private_test = private_test.assign(predictions=model.predict_structures(private_test.structures))
    # private_test[['predictions']].to_csv('./submission111.csv', index_label='id')

    private_test = pd.DataFrame(columns=['id', 'structures'], index=struct.index.values.tolist())
    private_test = private_test.assign(structures=struct.structures.values.tolist())
    private_test = private_test.assign(predictions=model.predict_structures(private_test.structures))
    private_test[['predictions']].to_csv('./submission111.csv', index_label='id')


In [ ]:
if __name__ == "__main__":
    with open("config.yaml") as file:
        config = yaml.safe_load(file)
    main_to_predict(config)